In [2]:
from langchain_community.document_loaders import TextLoader
docs = TextLoader("speech.txt").load()

In [3]:
# 2. Chunk documents
from langchain_text_splitters import CharacterTextSplitter
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [4]:
# 3. Choose an embedding model
from langchain_community.embeddings import HuggingFaceEmbeddings  # or OpenAIEmbeddings, GeminiEmbeddings, etc.
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

C:\Users\jayan\AppData\Local\Temp\ipykernel_13896\3690521581.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
d:\End_to_End_agentic_AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 4. Store in vector DB
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

In [7]:
# 5. Save DB (optional)
vectorstore.save_local("faiss_index")

In [8]:
# 6. Perform similarity search
query = "Explain non-violence"
results = vectorstore.similarity_search(query, k=3)

for r in results:
    print(r.page_content)

Occasions like the present do not occur in everybodyâ€™s and but rarely in anybodyâ€™s life.  
I want you to know and feel that there is nothing but purest Ahimsa in all that I am saying and doing today.  
The draft resolution of the Working Committee is based on Ahimsa; the contemplated struggle similarly has its roots in Ahimsa.  
If, therefore, there is any among you who has lost faith in Ahimsa or is wearied of it, let him not vote for this resolution...
Here is a mantra, a short one, that I give you. You may imprint it on your hearts and let every breath of yours give expression to it.  
The mantra is: **â€˜Do or Die.â€™** We shall either free India or die in the attempt; we shall not live to see the perpetuation of our slavery.  
Every true Congressman or woman will join the struggle with an inflexible determinationâ€¦


LLM POWERED RETRIEVAL

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

In [10]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)


In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # uses OpenAI-compatible APIs

llm = ChatOpenAI(
    temperature=0,
    model_name="mistralai/mixtral-8x7b-instruct",  # or other OpenRouter model
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

query = "What is the message of Gandhi's Do or Die speech?"
result = rag_chain.invoke(query)

print(result["result"])


C:\Users\jayan\AppData\Local\Temp\ipykernel_13896\728487311.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


 The message of Gandhi's "Do or Die" speech is one of determination and commitment to the cause of Indian independence, with a strong emphasis on non-violence or Ahimsa. Gandhi urges every Congressman and woman to join the struggle for independence with an inflexible determination, stating that they will either free India or die in the attempt. He makes it clear that this struggle is based on Ahimsa, and that it is not a call for violence or hatred. Rather, it is a call for self-sacrifice and dedication to the cause of freedom, even in the face of great personal risk. Gandhi also makes it clear that those who have lost faith in Ahimsa or are wearied of it should not support this resolution, emphasizing the importance of non-violence to the struggle.
